In [ ]:
def generate_cluster_labels(session):
    """Generate descriptive labels for clusters based on caption content."""
    labels = {}
    
    # Get sample captions for each cluster
    cluster_content = session.sql("""
        SELECT 
            c.CLUSTER_ID,
            ARRAY_AGG(s.CAPTION) WITHIN GROUP (ORDER BY RANDOM()) AS CAPTIONS
        FROM MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA s
        JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_EMBEDS e 
            ON s.CAPTION = e.CONTENT
        JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_CLUSTERS c 
            ON e.CONTENT_HASH = c.CONTENT_HASH
        GROUP BY c.CLUSTER_ID
    """).to_pandas()
    
    for _, row in cluster_content.iterrows():
        cluster_id = row['CLUSTER_ID']
        captions = row['CAPTIONS']
        
        # Take up to 10 sample captions
        sample_captions = captions[:10] if len(captions) > 10 else captions
        captions_text = "\n".join([f"- {c[:200]}" for c in sample_captions])
        
        prompt = f"""Analyze these social media captions from a content cluster and generate a short descriptive label (2-4 words) that captures the main theme or topic:

{captions_text}

Return only the label, nothing else."""
        
        result = session.sql(f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE('gemini-3-pro', $${prompt}$$) AS label
        """).collect()
        
        labels[cluster_id] = result[0]['LABEL'].strip()
    
    return labels

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# First, get your clustered data with numeric features
cluster_df = session.sql("""
    SELECT 
        c.CLUSTER_ID,
        s.LIKESCOUNT,
        s.COMMENTSCOUNT,
        s.LIKESCOUNT + s.COMMENTSCOUNT AS TOTAL_INTERACTIONS
    FROM MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA s
    JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_EMBEDS e 
        ON s.CAPTION = e.CONTENT
    JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_CLUSTERS c 
        ON e.CONTENT_HASH = c.CONTENT_HASH
""").to_pandas()

# Generate labels using numeric features
cluster_labels = generate_cluster_labels(session)

# After generating cluster_labels, create a DataFrame and save to Snowflake
labels_df = pd.DataFrame([
    {'CLUSTER_ID': k, 'CLUSTER_LABEL': v} 
    for k, v in cluster_labels.items()
])

# Write to Snowflake table
session.write_pandas(
    labels_df, 
    'CLUSTER_LABELS',
    database='MCWICS_HACKATHON_DATA',
    schema='PUBLIC',
    auto_create_table=True,
    overwrite=True
)

print("Cluster labels saved to MCWICS_HACKATHON_DATA.PUBLIC.CLUSTER_LABELS")

In [ ]:
-- Cell 1: Prepare data by joining with clusters
-- Get interaction data with cluster assignments
SELECT 
    s.TIMESTAMP,
    s.LIKESCOUNT,
    s.COMMENTSCOUNT,
    s.LIKESCOUNT + s.COMMENTSCOUNT AS TOTAL_INTERACTIONS,
    c.CLUSTER_ID
FROM MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA s
JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_EMBEDS e 
    ON s.CAPTION = e.CONTENT
JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_CLUSTERS c 
    ON e.CONTENT_HASH = c.CONTENT_HASH
JOIN MCWICS_HACKATHON_DATA.PUBLIC.CLUSTER_LABELS l
    ON l.cluster_id = c.cluster_id
ORDER BY c.CLUSTER_ID, s.TIMESTAMP

In [ ]:
# Aggregate to daily level per cluster
agg_query = """
SELECT 
    DATE_TRUNC('DAY', s.TIMESTAMP) AS DATE,
    c.CLUSTER_ID,
    SUM(s.LIKESCOUNT + s.COMMENTSCOUNT) AS TOTAL_INTERACTIONS
FROM MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA s
JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_EMBEDS e 
    ON s.CAPTION = e.CONTENT
JOIN MCWICS_HACKATHON_DATA.PUBLIC.SCRAPEDATA_CAPTION_CLUSTERS c 
    ON e.CONTENT_HASH = c.CONTENT_HASH
GROUP BY DATE_TRUNC('DAY', s.TIMESTAMP), c.CLUSTER_ID
ORDER BY c.CLUSTER_ID, DATE
"""

session.sql("CREATE OR REPLACE VIEW forecast_input AS " + agg_query).collect()

# Add ON_ERROR => 'SKIP' to skip clusters with insufficient data
session.sql("""
CREATE OR REPLACE SNOWFLAKE.ML.FORECAST interaction_forecast_model(
    INPUT_DATA => TABLE(forecast_input),
    SERIES_COLNAME => 'CLUSTER_ID',
    TIMESTAMP_COLNAME => 'DATE',
    TARGET_COLNAME => 'TOTAL_INTERACTIONS',
    CONFIG_OBJECT => {'ON_ERROR': 'SKIP'}
)
""").collect()

print("Forecast model trained successfully!")

# Check which series failed
session.sql("CALL interaction_forecast_model!SHOW_TRAINING_LOGS()").show()

In [ ]:
# Analyze forecast results and generate LLM insights with cluster labels
import json
from datetime import datetime

# Get forecast results
forecast_results = session.sql("""
SELECT * FROM TABLE(interaction_forecast_model!FORECAST(FORECASTING_PERIODS => 7))
ORDER BY SERIES, TS
""").to_pandas()

# Get forecast summary stats per cluster
forecast_summary = forecast_results.groupby('SERIES').agg({
    'FORECAST': ['mean', 'sum', 'max'],
    'UPPER_BOUND': 'max',
    'LOWER_BOUND': 'min'
}).round(2)

forecast_summary.columns = ['avg_forecast', 'total_forecast', 'peak_forecast', 'upper_bound', 'lower_bound']
forecast_summary = forecast_summary.reset_index()

# Get cluster labels
labels_df = session.sql("""
    SELECT CLUSTER_ID, CLUSTER_LABEL 
    FROM MCWICS_HACKATHON_DATA.PUBLIC.CLUSTER_LABELS
""").to_pandas()

# Convert both columns to same type before merge
forecast_summary['SERIES'] = forecast_summary['SERIES'].astype(int)
labels_df['CLUSTER_ID'] = labels_df['CLUSTER_ID'].astype(int)

forecast_summary = forecast_summary.merge(labels_df, left_on='SERIES', right_on='CLUSTER_ID', how='left')
forecast_summary['CLUSTER_LABEL'] = forecast_summary['CLUSTER_LABEL'].fillna(forecast_summary['SERIES'].apply(lambda x: f"Cluster {x}"))

# Build context for LLM with cluster labels
summary_text = "Forecasted engagement by content cluster (next 7 days):\n\n"
for _, row in forecast_summary.iterrows():
    summary_text += f"- {row['CLUSTER_LABEL']} (ID {row['SERIES']}): avg={row['avg_forecast']:.0f}, total={row['total_forecast']:.0f}, peak={row['peak_forecast']:.0f}\n"

prompt = f"""You are a social media strategist. Analyze this engagement forecast data and provide actionable insights.

The clusters represent different types of social media content based on their captions and engagement patterns:

{summary_text}

Answer these questions:
1. Which content type shows the highest predicted engagement? Reference by its label name.
2. Based on the cluster labels, what themes or content strategies are working best?
3. What specific recommendations would you give to maximize community engagement?
4. Which content types should be deprioritized and why?

Reference cluster labels by name, and provide actionable recommendations.
Be short and concise so that the reader can understand the insights quickly."""

# Generate insights using LLM
insights_result = session.sql(f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE('gemini-3-pro', $${prompt}$$) AS insights
""").collect()

insights = insights_result[0]['INSIGHTS']

# Save insights to table
insights_df = pd.DataFrame([{
    'GENERATED_AT': datetime.now(),
    'INSIGHTS': insights,
    'FORECAST_SUMMARY': summary_text,
    'NUM_CLUSTERS': len(forecast_summary)
}])

session.write_pandas(
    insights_df,
    'ENGAGEMENT_INSIGHTS',
    database='MCWICS_HACKATHON_DATA',
    schema='PUBLIC',
    auto_create_table=True,
    overwrite=True
)

print("Insights saved to MCWICS_HACKATHON_DATA.PUBLIC.ENGAGEMENT_INSIGHTS")

# Display results
st.subheader("AI-Generated Engagement Insights")
st.markdown(insights)

# Show summary table with labels
st.subheader("Forecast Summary by Content Type")
display_cols = ['CLUSTER_LABEL', 'avg_forecast', 'total_forecast', 'peak_forecast']
st.dataframe(forecast_summary[display_cols].sort_values('total_forecast', ascending=False))